# BLIP QA Automated Test Suite

This notebook runs a full suite of tests against the landmark QA dataset using the `run_tests.py` script. It covers:
1.  **Positive Tests:** Testing each landmark's questions against its own image.
2.  **Negative Tests:** Testing confusing pairs of landmarks against each other to check for distinctiveness.

The results are collected and displayed in summary tables using pandas.

## 1. Setup and Configuration

In [2]:
# 로컬/서버에서 실행할 땐 아래 코드 주석처리 하세요
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
import json
import pandas as pd
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
import warnings

# --- 1. 환경별 프로젝트 루트 경로 정의 ---
COLAB_DRIVE_ROOT = "/content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github"
SERVER_ROOT = "/content/group5_project"

# --- 2. 환경 감지 및 PROJECT_ROOT 설정 ---
if os.path.exists(COLAB_DRIVE_ROOT):
    PROJECT_ROOT = COLAB_DRIVE_ROOT
    print("환경 감지: Google Drive Colab")
elif os.path.exists(SERVER_ROOT):
    PROJECT_ROOT = SERVER_ROOT
    print("환경 감지: Server (group5_project)")
else:
    PROJECT_ROOT = os.getcwd()
    print("환경 감지: Local workspace")

# --- Configuration ---
QA_FILE = os.path.join(PROJECT_ROOT, "data", "landmark_qa_labeled_updated.json")
DATA_ROOT = os.path.join(PROJECT_ROOT, "data")

warnings.filterwarnings("ignore", message=".*TypedStorage is deprecated.*")

with open(QA_FILE, 'r', encoding='utf-8') as f:
    QA_DATA_SNAPSHOT = json.load(f)
LANDMARK_NAMES = list(QA_DATA_SNAPSHOT.keys())
print(f"QA landmarks loaded: {len(LANDMARK_NAMES)} entries")

IMAGE_EXTENSIONS = (".jpg", ".jpeg", ".png", ".jfif", ".heic", ".webp")


def collect_landmark_images(data_root, landmark_names, max_images=5):
    image_map = {}
    for landmark in landmark_names:
        folder = os.path.join(data_root, landmark)
        images = []
        if os.path.isdir(folder):
            for root, _, files in os.walk(folder):
                for filename in sorted(files):
                    if filename.lower().endswith(IMAGE_EXTENSIONS):
                        rel_path = os.path.relpath(os.path.join(root, filename), data_root)
                        images.append(rel_path)
        if not images:
            print(f"[경고] '{landmark}' 폴더에서 사용할 이미지가 없습니다.")
        image_map[landmark] = images[:max_images]
    return image_map


landmark_images = collect_landmark_images(DATA_ROOT, LANDMARK_NAMES, max_images=5)
print("Setup complete.")
print(f"QA File: {QA_FILE}")
print(f"Data Root: {DATA_ROOT}")
print(f"Representative image pools ready for {len(landmark_images)} landmarks (≤5 each)")



환경 감지: Google Drive Colab
QA landmarks loaded: 11 entries
Setup complete.
QA File: /content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github/data/landmark_qa_labeled_updated.json
Data Root: /content/drive/Othercomputers/내 컴퓨터/데이콘 출판마을 프로젝트/github/data
Representative image pools ready for 11 landmarks (≤5 each)


## 2. BLIP Model Setup


In [4]:
processor = None
model = None
device = "cuda" if torch.cuda.is_available() else "cpu"


def load_blip_model():
    global processor, model
    if processor is None or model is None:
        processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
        model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
        model.to(device)
        print(f"BLIP VQA model loaded on device: {device}")
    return processor, model


def run_vqa_test(qa_file_path, landmark_name, image_path):
    processor, model = load_blip_model()

    try:
        with open(qa_file_path, 'r', encoding='utf-8') as f:
            qa_data = json.load(f)
    except FileNotFoundError:
        return {"error": f"QA 파일을 찾을 수 없습니다: {qa_file_path}"}

    if landmark_name not in qa_data:
        return {"error": f"랜드마크 '{landmark_name}' 데이터를 찾을 수 없습니다."}

    try:
        raw_image = Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        return {"error": f"이미지 파일을 찾을 수 없습니다: {image_path}"}

    questions_for_landmark = qa_data[landmark_name]
    correct_yes_count = 0
    total_yes_questions = 0
    correct_no_count = 0
    total_no_questions = 0

    print(f"\n--- Running Evaluation for '{landmark_name}' with image '{os.path.basename(image_path)}' ---")
    for i, (question, expected_answer) in enumerate(questions_for_landmark):
        inputs = processor(raw_image, question, return_tensors="pt").to(device)
        out = model.generate(**inputs, max_new_tokens=10)
        model_answer = processor.decode(out[0], skip_special_tokens=True).strip().lower()

        is_correct = (model_answer == expected_answer)
        if expected_answer == 'yes':
            total_yes_questions += 1
            if is_correct:
                correct_yes_count += 1
        else:
            total_no_questions += 1
            if is_correct:
                correct_no_count += 1

        print(f"Q{i+1:02d}: '{question}' -> Model: '{model_answer}', Expected: '{expected_answer}' ({'Correct' if is_correct else 'Incorrect'})")

    yes_accuracy = (correct_yes_count / total_yes_questions) if total_yes_questions else 0
    no_accuracy = (correct_no_count / total_no_questions) if total_no_questions else 0
    total_accuracy = ((correct_yes_count + correct_no_count) / (total_yes_questions + total_no_questions)) if (total_yes_questions + total_no_questions) else 0

    print("\n--- Test Summary ---")
    print(f"YES Questions Accuracy: {correct_yes_count}/{total_yes_questions} ({yes_accuracy:.2%})")
    print(f"NO Questions Accuracy: {correct_no_count}/{total_no_questions} ({no_accuracy:.2%})")
    print(f"Overall Accuracy: {total_accuracy:.2%}")
    print("-" * 20)

    return {
        "yes_accuracy": yes_accuracy * 100,
        "no_accuracy": no_accuracy * 100,
        "overall_accuracy": total_accuracy * 100
    }



## 3. Helper Function to Run Tests


In [5]:
def run_test(landmark_name, image_name, image_path):
    """Run BLIP QA locally and return accuracy metrics."""
    print(f"--- Running test: Questions for [{landmark_name}] with image of [{image_name}] ---")
    result = run_vqa_test(QA_FILE, landmark_name, image_path)

    if "error" in result:
        print(result["error"])
        return {"error": result["error"]}

    return {
        "YES Accuracy": round(result["yes_accuracy"], 2),
        "NO Accuracy": round(result["no_accuracy"], 2),
        "Overall Accuracy": round(result["overall_accuracy"], 2)
    }



## 4. Execute Positive Tests
각 랜드마크의 대표 이미지 최대 3장을 사용해 테스트를 반복 실행하고, 평균 정확도를 계산합니다.


In [6]:
positive_results = []

for landmark in LANDMARK_NAMES:
    image_rel_paths = landmark_images.get(landmark, [])
    sample_metrics = []

    for rel_path in image_rel_paths:
        image_path = os.path.join(DATA_ROOT, rel_path)
        accuracy_data = run_test(landmark, landmark, image_path)
        if "error" in accuracy_data:
            continue
        sample_metrics.append(accuracy_data)

    if sample_metrics:
        mean_yes = sum(m["YES Accuracy"] for m in sample_metrics) / len(sample_metrics)
        mean_no = sum(m["NO Accuracy"] for m in sample_metrics) / len(sample_metrics)
        mean_overall = sum(m["Overall Accuracy"] for m in sample_metrics) / len(sample_metrics)
    else:
        mean_yes = mean_no = mean_overall = None

    positive_results.append({
        "Landmark": landmark,
        "YES Accuracy (mean)": round(mean_yes, 2) if mean_yes is not None else None,
        "NO Accuracy (mean)": round(mean_no, 2) if mean_no is not None else None,
        "Overall Accuracy (mean)": round(mean_overall, 2) if mean_overall is not None else None,
        "Samples": len(sample_metrics)
    })
    print("-" * 50)

print("=== Positive Tests Complete ===")
positive_df = pd.DataFrame(positive_results).set_index("Landmark")
positive_df



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


--- Running test: Questions for [마법천자문 손오공] with image of [마법천자문 손오공] ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

from matplotlib import pyplot as plt
positive_df['YES Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='YES Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 47560 (\N{HANGUL SYLLABLE MA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 48277 (\N{HANGUL SYLLABLE BEOB}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52380 (\N{HANGUL SYLLABLE CEON}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 47928 (\N{HANGUL SYLLABLE MUN}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 49552 (\N{HANGUL SYLLABLE

from matplotlib import pyplot as plt
positive_df['NO Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='NO Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 47560 (\N{HANGUL SYLLABLE MA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 48277 (\N{HANGUL SYLLABLE BEOB}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52380 (\N{HANGUL SYLLABLE CEON}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 47928 (\N{HANGUL SYLLABLE MUN}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.12/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 49552 (\N{HANGUL SYLLABLE

from matplotlib import pyplot as plt
positive_df['Overall Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='Overall Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

## 5. Execute Negative Tests
모든 랜드마크 조합(자기 자신 제외)에 대해 상대 랜드마크 이미지 3장을 사용하여 혼동 여부를 평균 정확도로 확인합니다.


In [7]:
negative_results = []

for questions_for in LANDMARK_NAMES:
    for image_of in LANDMARK_NAMES:
        if questions_for == image_of:
            continue

        image_rel_paths = landmark_images.get(image_of, [])
        sample_metrics = []

        for rel_path in image_rel_paths:
            image_path = os.path.join(DATA_ROOT, rel_path)
            accuracy_data = run_test(questions_for, image_of, image_path)
            if "error" in accuracy_data:
                continue
            sample_metrics.append(accuracy_data)

        if sample_metrics:
            mean_yes = sum(m["YES Accuracy"] for m in sample_metrics) / len(sample_metrics)
            mean_no = sum(m["NO Accuracy"] for m in sample_metrics) / len(sample_metrics)
            mean_overall = sum(m["Overall Accuracy"] for m in sample_metrics) / len(sample_metrics)
        else:
            mean_yes = mean_no = mean_overall = None

        negative_results.append({
            "Questions For": questions_for,
            "Image Of": image_of,
            "YES Accuracy (mean)": round(mean_yes, 2) if mean_yes is not None else None,
            "NO Accuracy (mean)": round(mean_no, 2) if mean_no is not None else None,
            "Overall Accuracy (mean)": round(mean_overall, 2) if mean_overall is not None else None,
            "Samples": len(sample_metrics)
        })
        print("-" * 50)

print("=== Negative Tests Complete ===")
negative_df = pd.DataFrame(negative_results)
negative_df



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Q18: 'Are the characters standing on small circular bases?' -> Model: 'yes', Expected: 'yes' (Correct)
Q19: 'Is this a group of colorful statues?' -> Model: 'yes', Expected: 'yes' (Correct)
Q20: 'Is there a building in the background?' -> Model: 'yes', Expected: 'yes' (Correct)
Q21: 'Is the tail red?' -> Model: 'no', Expected: 'no' (Correct)
Q22: 'Is the teardrop coming from one eye?' -> Model: 'yes', Expected: 'no' (Incorrect)
Q23: 'Does the Blue Fairy statue have blue hair?' -> Model: 'yes', Expected: 'no' (Incorrect)
Q24: 'Are the sculpture's legs hanging down?' -> Model: 'yes', Expected: 'no' (Incorrect)
Q25: 'Is the statue's main color pink?' -> Model: 'no', Expected: 'no' (Correct)
Q26: 'Is the main object a sculpture?' -> Model: 'yes', Expected: 'no' (Incorrect)
Q27: 'Is the building's surface textured by the leaves?' -> Model: 'yes', Expected: 'no' (Incorrect)
Q28: 'Is the bird an owl?' -> Model: 'no', Expected: 'no' (Correct)
Q29: 'Does the 

,Questions For,Image Of,YES Accuracy (mean),NO Accuracy (mean),Overall Accuracy (mean),Samples
0,마법천자문 손오공,로드킬 부엉이,19.0,54.0,36.5,5
1,마법천자문 손오공,나남출판사,18.0,64.0,41.0,5
2,마법천자문 손오공,활판공방 인쇄기,26.0,46.0,36.0,5
3,마법천자문 손오공,활돌이,39.0,47.0,43.0,5
4,마법천자문 손오공,지혜의숲 고양이,26.0,37.0,31.5,5
...,...,...,...,...,...,...
105,피노지움 조각상,지혜의숲 고양이,48.0,70.0,59.0,5
106,피노지움 조각상,네모탑,30.0,75.0,52.5,5
107,피노지움 조각상,지혜의숲 조각상,53.0,45.0,49.0,5
108,피노지움 조각상,지혜의숲 입구 조각상,57.0,34.0,45.5,5


from matplotlib import pyplot as plt
negative_df['YES Accuracy (mean)'].plot(kind='hist', bins=20, title='YES Accuracy (mean)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
negative_df['NO Accuracy (mean)'].plot(kind='hist', bins=20, title='NO Accuracy (mean)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
negative_df['Overall Accuracy (mean)'].plot(kind='hist', bins=20, title='Overall Accuracy (mean)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Samples']
  ys = series['Overall Accuracy (mean)']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = negative_df.sort_values('Samples', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Samples')
_ = plt.ylabel('Overall Accuracy (mean)')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Samples']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Samples'}, axis=1)
              .sort_values('Samples', ascending=True))
  xs = counted['Samples']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = negative_df.sort_values('Samples', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Samples')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
negative_df['YES Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='YES Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
negative_df['NO Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='NO Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
negative_df['Overall Accuracy (mean)'].plot(kind='line', figsize=(8, 4), title='Overall Accuracy (mean)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [12]:
pd.set_option('display.max_rows', None)

In [13]:
negative_df

,Questions For,Image Of,YES Accuracy (mean),NO Accuracy (mean),Overall Accuracy (mean),Samples
0,마법천자문 손오공,로드킬 부엉이,19.0,54.0,36.5,5
1,마법천자문 손오공,나남출판사,18.0,64.0,41.0,5
2,마법천자문 손오공,활판공방 인쇄기,26.0,46.0,36.0,5
3,마법천자문 손오공,활돌이,39.0,47.0,43.0,5
4,마법천자문 손오공,지혜의숲 고양이,26.0,37.0,31.5,5
5,마법천자문 손오공,네모탑,19.0,58.0,38.5,5
6,마법천자문 손오공,지혜의숲 조각상,57.0,31.0,44.0,5
7,마법천자문 손오공,지혜의숲 입구 조각상,62.0,43.0,52.5,5
8,마법천자문 손오공,창틀 피노키오,51.0,64.0,57.5,5
9,마법천자문 손오공,피노지움 조각상,76.0,23.0,49.5,5


In [11]:
for landmark in LANDMARK_NAMES:
    negative_df[negative_df['Questions For'] == landmark]

## 6. Results Summary


In [14]:
print("--- Positive Test Results ---")
display(positive_df)

print("\n--- Negative Test Results ---")
display(negative_df)

--- Positive Test Results ---


,YES Accuracy (mean),NO Accuracy (mean),Overall Accuracy (mean),Samples
Landmark,,,,
마법천자문 손오공,92.0,69.0,80.5,5
로드킬 부엉이,93.0,69.0,81.0,5
나남출판사,90.0,69.0,79.5,5
활판공방 인쇄기,95.0,57.0,76.0,5
활돌이,85.0,54.0,69.5,5
지혜의숲 고양이,97.0,59.0,78.0,5
네모탑,87.0,68.0,77.5,5
지혜의숲 조각상,97.0,40.0,68.5,5
지혜의숲 입구 조각상,98.0,52.0,75.0,5



--- Negative Test Results ---


,Questions For,Image Of,YES Accuracy (mean),NO Accuracy (mean),Overall Accuracy (mean),Samples
0,마법천자문 손오공,로드킬 부엉이,19.0,54.0,36.5,5
1,마법천자문 손오공,나남출판사,18.0,64.0,41.0,5
2,마법천자문 손오공,활판공방 인쇄기,26.0,46.0,36.0,5
3,마법천자문 손오공,활돌이,39.0,47.0,43.0,5
4,마법천자문 손오공,지혜의숲 고양이,26.0,37.0,31.5,5
5,마법천자문 손오공,네모탑,19.0,58.0,38.5,5
6,마법천자문 손오공,지혜의숲 조각상,57.0,31.0,44.0,5
7,마법천자문 손오공,지혜의숲 입구 조각상,62.0,43.0,52.5,5
8,마법천자문 손오공,창틀 피노키오,51.0,64.0,57.5,5
9,마법천자문 손오공,피노지움 조각상,76.0,23.0,49.5,5


## 7. Analysis
- Use the positive accuracy table to highlight landmarks where YES accuracy falls below target; add images or clarify hints/questions accordingly.
- In the negative table, any case with non-zero YES accuracy indicates confusing phrasing—add more discriminative NO samples for that landmark pair.
- Document follow-up changes and rerun this notebook so new accuracy numbers serve as regression checkpoints.
